In [2]:
import pandas as pd
import numpy as np


In [3]:
songs=pd.read_csv("songs.csv")

In [4]:
songs=songs.drop('link',axis=1).reset_index(drop=True)

In [5]:
songs

,artist,song,text
0,ABBA,Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante","Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,Making somebody happy is a question of give an...
...,...,...,...
57645,Ziggy Marley,Good Old Days,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,northern star \r\nam i frightened \r\nwhere ...


Preprocessing


In [6]:
songs['text']=songs['text'].str.lower().str.replace('\r', '').str.replace('\n', '')

In [7]:
# songs['text']=songs['text'].str.replace('\\r', '').str.replace('\\n', '')
songs['song']=songs['song'].str.lower()

In [8]:
# songs=songs.sample(5000).reset_index(drop=True)

In [9]:
songs['recommend'] = None

In [10]:
songs

,artist,song,text,recommend
0,ABBA,ahe's my kind of girl,"look at her face, it's a wonderful face and i...",None
1,ABBA,"andante, andante","take it easy with me, please touch me gently ...",None
2,ABBA,as good as new,i'll never know why i had to go why i had to ...,None
3,ABBA,bang,making somebody happy is a question of give an...,None
4,ABBA,bang-a-boomerang,making somebody happy is a question of give an...,None
...,...,...,...,...
57645,Ziggy Marley,good old days,irie days come on play let the angels fly let...,None
57646,Ziggy Marley,hand to mouth,power to the workers more power power to the...,None
57647,Zwan,come with me,all you need is something i'll believe flash...,None
57648,Zwan,desire,northern star am i frightened where can i go...,None


In [11]:
import nltk
from nltk.stem.porter import PorterStemmer

In [12]:
stemmer=PorterStemmer()

In [13]:
def token(elm):
  tok=nltk.word_tokenize(elm)
  a=[stemmer.stem(x) for x in tok]
  return ' '.join(a)

In [14]:
songs['text']=songs['text'].apply(lambda x: token(x))

In [15]:
songs['artist']=songs['artist'].str.lower()

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
tfid=TfidfVectorizer(analyzer='word', stop_words='english')

In [18]:
matrix=tfid.fit_transform(songs['text'])

In [19]:
similarity=cosine_similarity(matrix)

MemoryError: Unable to allocate 23.7 GiB for an array with shape (3174427066,) and data type int64

In [ ]:
similarity[0]

Recommender function

In [ ]:
def recommender(song_name):
    try:
        i = songs[songs['song'] == song_name].index[0]
    except IndexError:
        # Handle the case where the song doesn't exist
        return ["Song not found. Please provide a valid song name."]

    distances = sorted(enumerate(similarity[i]), key=lambda x: x[1], reverse=True)
    song_indices = [index for index, _ in distances[1:11]]  # Exclude the input song itself

    recommended_songs = []
    for index in song_indices:
        song = songs.loc[index, 'song']
        artist = songs.loc[index, 'artist']
        recommended_songs.append(song + '   ' + artist)

    return recommended_songs


In [ ]:
for index, row in songs.iterrows():
    song_name = row['song']
    recommendations = recommender(song_name)
    songs.at[index, 'recommend'] = recommendations

In [ ]:
songs['recommend'].iloc[84]

In [ ]:
songs

In [ ]:
import json

data_to_export = {
    "songs": songs.to_dict(orient="records")
}

with open('songsmodel.json', 'w') as file:
    json.dump(data_to_export, file)

In [25]:
# recommendation = recommender('alone')
# print(recommendation)